In [1]:
import os
import sqlite3 
import numpy as np
import pandas as pd
import rubin_sim.maf as maf
from rubin_scheduler.utils import Site
from rubin_scheduler.site_models import Almanac

from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
from astroplan import Observer, FixedTarget
from rubin_scheduler.scheduler.utils import smallest_signed_angle
from rubin_scheduler.utils import Site

In [2]:
opsdb = 'ops_rehearsal_jun_2024_v1.db'

In [3]:
conn = sqlite3.connect(opsdb)
query = 'select * from observations'

visits = pd.read_sql(query, conn)

In [11]:
visits[['observationStartMJD', 'fieldRA', 'fieldDec', 'rotTelPos', 'paraAngle', 'rotSkyPos']]

,observationStartMJD,fieldRA,fieldDec,rotTelPos,paraAngle,rotSkyPos
0,60474.949825,224.885530,-39.966701,0.00000,266.598569,93.401431
1,60474.950222,224.894792,-39.881273,1.00000,266.860386,94.139614
2,60474.950620,225.120458,-39.895901,2.00000,266.827462,95.172538
3,60474.951019,225.138319,-40.091449,3.00000,267.118622,95.881378
4,60474.951417,224.918541,-40.060545,4.00000,267.303274,96.696726
...,...,...,...,...,...,...
22735,60495.443601,309.896018,-19.172486,36.57233,114.182721,282.389609
22736,60495.443996,309.888374,-19.133497,37.57233,114.217285,283.355044
22737,60495.445802,58.663126,-49.330231,37.57233,281.077523,116.494806
22738,60495.446201,58.808230,-49.136984,38.57233,281.309362,117.262968


In [5]:
lsst = Site('LSST')
observer = Observer(longitude=lsst.longitude*u.deg, latitude=lsst.latitude*u.deg,
                    elevation=lsst.height*u.m, name="LSST", 
                    timezone="Chile/Continental", 
                    pressure=lsst.pressure*u.mBa, 
                    temperature=lsst.temperature*u.deg_C, 
                    relative_humidity=lsst.humidity)

In [6]:
print('LSST site')
print('lat', lsst.latitude*u.deg)
print('lon', lsst.longitude*u.deg)
print('elevation', lsst.height*u.m)
print('pressure', lsst.pressure*u.mBa)
print('temperature', lsst.temperature*u.deg_C)
print('humidity', lsst.humidity)

LSST site
lat -30.2444 deg
lon -70.7494 deg
elevation 2650.0 m
pressure 750.0 mBa
temperature 11.5 deg_C
humidity 0.4


In [7]:
coords = SkyCoord(visits['fieldRA'].values, visits['fieldDec'].values, unit='deg', frame='icrs')
targets = FixedTarget(coords)
times = Time(visits['observationStartMJD'], format='mjd', scale='tai')
altaz = observer.altaz(times, targets)
pa = observer.parallactic_angle(times, targets, kind='apparent')
pa = pa.deg % 360
skyrot = (pa - visits['rotTelPos'].values - 90) % 360

In [8]:
visits['newPA'] = pa
visits['newRotSkyPos'] = skyrot

In [9]:
visits[['observationStartMJD', 'fieldRA', 'fieldDec', 'rotTelPos', 'paraAngle', 'newPA', 'rotSkyPos', 'newRotSkyPos']]

,observationStartMJD,fieldRA,fieldDec,rotTelPos,paraAngle,newPA,rotSkyPos,newRotSkyPos
0,60474.949825,224.885530,-39.966701,0.00000,266.598569,266.790232,93.401431,176.790232
1,60474.950222,224.894792,-39.881273,1.00000,266.860386,266.776078,94.139614,175.776078
2,60474.950620,225.120458,-39.895901,2.00000,266.827462,266.743613,95.172538,174.743613
3,60474.951019,225.138319,-40.091449,3.00000,267.118622,267.033717,95.881378,174.033717
4,60474.951417,224.918541,-40.060545,4.00000,267.303274,267.218171,96.696726,173.218171
...,...,...,...,...,...,...,...,...
22735,60495.443601,309.896018,-19.172486,36.57233,114.182721,114.172305,282.389609,347.599976
22736,60495.443996,309.888374,-19.133497,37.57233,114.217285,114.206748,283.355044,346.634419
22737,60495.445802,58.663126,-49.330231,37.57233,281.077523,281.412455,116.494806,153.840126
22738,60495.446201,58.808230,-49.136984,38.57233,281.309362,281.180921,117.262968,152.608592
